In [107]:
import pandas as pd
import numpy as np

In [108]:
data = pd.read_excel('scores.xlsx', index_col=0)

In [109]:
factors = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12']
indices = ['V','M','I']

In [110]:
data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12
V,0.4055,0.8640,0.8640,0.928187,0.7520,0.928687,0.8645,0.8640,0.928687,0.7555,0.6675,0.8640
M,0.6395,0.2445,0.2445,0.244062,0.3960,0.135562,0.1360,0.2445,0.135562,0.2480,0.4055,0.2445
I,0.1355,0.0100,0.0100,0.010000,0.1355,0.010000,0.0100,0.0100,0.010000,0.0100,0.1355,0.0100


In [111]:
def calculate_uij(xij, L, M, H):
    """
    Calculate μij based on the given conditions for xij.
    
    Parameters:
    xij (float): Input value
    M (float): Parameter M
    H (float): Parameter H
    L (float): Parameter L
    
    Returns:
    tuple: The calculated μij tuple based on the conditions
    """
    # if xij ∈ [M,H] and xij ≥ (M + H)/2
    if M <= xij <= H and xij >= (M + H)/2:
        return (xij, xij, (H + xij)/2)
    
    # if xij = H
    if xij == H:
        return ((3*H + M)/4, H, H)
    
    # if xij = M
    if xij == M:
        return ((M - L) - M**3/(3*H), M, (3*H*L + M**2)/M)
    
    # if xij = L
    if xij == L:
        return (L, L, L)
    
    # if xij < M
    if xij < M:
        return (((xij + 2*L + M)/2)*xij, xij, xij)
    
    # if xij > M and xij < (M + H)/2
    if xij > M and xij < (M + H)/2:
        return ((3*xij + M)/4, xij, xij)
    
    # If none of the conditions are met, return None or raise an exception
    raise ValueError("Input xij does not meet any of the defined conditions")

In [112]:
L = 0.01
M = 0.5
H = 0.99

In [113]:
result = np.array([calculate_uij(x, L, M, H) for row in data.to_numpy() for x in row]).reshape(3, 12, 3)

In [114]:
result

array([[[0.18764512, 0.4055    , 0.4055    ],
        [0.864     , 0.864     , 0.927     ],
        [0.864     , 0.864     , 0.927     ],
        [0.9281875 , 0.9281875 , 0.95909375],
        [0.752     , 0.752     , 0.871     ],
        [0.9286875 , 0.9286875 , 0.95934375],
        [0.8645    , 0.8645    , 0.92725   ],
        [0.864     , 0.864     , 0.927     ],
        [0.9286875 , 0.9286875 , 0.95934375],
        [0.7555    , 0.7555    , 0.87275   ],
        [0.625625  , 0.6675    , 0.6675    ],
        [0.864     , 0.864     , 0.927     ]],

       [[0.604625  , 0.6395    , 0.6395    ],
        [0.09346012, 0.2445    , 0.2445    ],
        [0.09346012, 0.2445    , 0.2445    ],
        [0.0932395 , 0.2440625 , 0.2440625 ],
        [0.181368  , 0.396     , 0.396     ],
        [0.04443485, 0.1355625 , 0.1355625 ],
        [0.044608  , 0.136     , 0.136     ],
        [0.09346012, 0.2445    , 0.2445    ],
        [0.04443485, 0.1355625 , 0.1355625 ],
        [0.095232  , 0.248     ,

In [115]:
w1 = 0.6
w2 = 0.3
w3 = 0.1

w1n = w1
w2n = w2 
w3n = w3 

result[0] = w1n * result[0]
result[1] = w2n * result[1]
result[2] = w3n * result[2]
some = np.zeros((result.shape[0], result.shape[1]))
for i in range(result.shape[0]):
    for j in range(result.shape[1]):
        some[i][j] = result[i][j][0]*result[i][j][1] + result[i][j][1]*result[i][j][2] + result[i][j][2]*result[i][j][0]
aw = some.sum(axis=0)
# normalize the result
cw = aw / aw.sum()

In [116]:
# save to csv
result_df = pd.DataFrame({'SCR area':aw, 'Factor weight':cw})
result_df.index = factors

In [118]:
result_df.to_csv('result.csv', index=True, header=True)